# Cleaning Greenburgh, NY Police Data Sets

In [1]:
import os

import geocoder
import numpy as np
import pandas as pd
from pyproj import Proj, transform

In [2]:
directory = '/Users/VincentLa/git/greenburgh-town-supervisor-2019/src/data/school/budget/schooldistrict_all_years/'
output_file = '/Users/VincentLa/git/greenburgh-town-supervisor-2019/output/school/greenburgh_school_districts_budget.csv'

In [3]:
greenburgh_school_districts = [
    'Ardsley Union Free School District',
    'Dobbs Ferry Union Free School District',
    'Edgemont Union Free School District',
    'Elmsford Union Free School District',
    'Greenburgh Central School District',
    'Hastings-On-Hudson Union Free School District',
    'Irvington Union Free School District',
    'Union Free School District Of The Tarrytowns',
]

## Combining Data

In [4]:
budget_all_years = []
for file in os.listdir(directory):
    if file.endswith('.csv'):
        df = pd.read_csv(directory + file)
        df = df.loc[df.ENTITY_NAME.isin(greenburgh_school_districts)]
        df.reset_index(inplace=True)
        budget_all_years.append(df)

In [5]:
# Checking All School Districts exist in the dataset (none dropped out due to bad mismatch in names)
for df in budget_all_years:
    print(df.ENTITY_NAME.value_counts().shape)

(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)


In [6]:
df = pd.concat(budget_all_years, ignore_index=True)

In [7]:
df['Fiscal Year'] = df.CALENDAR_YEAR.map(str) + ' - ' + (df.CALENDAR_YEAR + 1).map(str)

In [8]:
df['LEVEL_1_CATEGORY'].value_counts()

EDUCATION                              17251
GENERAL GOVERNMENT                      7818
EMPLOYEE BENEFITS                       2193
STATE AID                               1688
USE AND SALE OF PROPERTY                1413
OTHER LOCAL REVENUES                    1317
DEBT SERVICE                            1199
FEDERAL AID                             1168
CHARGES FOR SERVICES                     499
OTHER USES                               474
OTHER SOURCES                            455
CHARGES TO OTHER GOVERNMENTS             292
OTHER REAL PROPERTY TAX ITEMS            287
SALES AND USE TAX                        197
REAL PROPERTY TAXES AND ASSESSMENTS      192
PROCEEDS OF DEBT                         171
CULTURE AND RECREATION                   156
TRANSPORTATION                           139
COMMUNITY SERVICES                       104
SOCIAL SERVICES                            3
Name: LEVEL_1_CATEGORY, dtype: int64

In [9]:
df.loc[pd.isnull(df['LEVEL_1_CATEGORY']), 'LEVEL_1_CATEGORY']

48       NaN
60       NaN
137      NaN
138      NaN
139      NaN
        ... 
50354    NaN
50355    NaN
50356    NaN
50357    NaN
50358    NaN
Name: LEVEL_1_CATEGORY, Length: 13343, dtype: object

In [10]:
# When Level 1 and 2 Category is Null then take the account code narrative
df.loc[pd.isnull(df['LEVEL_1_CATEGORY']), 'LEVEL_1_CATEGORY'] = df.loc[pd.isnull(df['LEVEL_1_CATEGORY']), 'ACCOUNT_CODE_NARRATIVE']
df.loc[pd.isnull(df['LEVEL_2_CATEGORY']), 'LEVEL_2_CATEGORY'] = df.loc[pd.isnull(df['LEVEL_2_CATEGORY']), 'ACCOUNT_CODE_NARRATIVE']

In [11]:
df.rename(columns={
    'CALENDAR_YEAR': 'Year',
    'MUNICIPAL_CODE':  'Municipal Code',
    'ENTITY_NAME': 'School District',
    'CLASS_DESCRIPTION': 'Class Description',
    'COUNTY': 'County',
    'ACCOUNT_CODE': 'Account Code',
    'ACCOUNT_CODE_NARRATIVE': 'Account Code Narrative',
    'FINANCIAL_STATEMENT': 'Financial Statement',
    'FINANCIAL_STATEMENT_SEGMENT': 'Financial Statement Segment',
    'LEVEL_1_CATEGORY': 'Level 1 Category',
    'LEVEL_2_CATEGORY': 'Level 2 Category',
    'OBJECT_OF_EXPENDITURE': 'Object of Expenditure',
    'AMOUNT': 'Amount',
}, inplace=True)

In [12]:
df.head()

,index,Year,Municipal Code,School District,Class Description,County,FISCAL_YEAR_END,Account Code,Account Code Narrative,Financial Statement,Financial Statement Segment,Level 1 Category,Level 2 Category,Object of Expenditure,Amount,SNAPSHOT_DATE,Fiscal Year
0,93783,2001,550634200500,Ardsley Union Free School District,School District - Independent Superintendent,Westchester,06/30,A1001,Real Property Taxes,STATEMENT OF REVENUES AND OTHER SOURCES,REVENUES,REAL PROPERTY TAXES AND ASSESSMENTS,REAL PROPERTY TAXES,NaN,17965391.0,31-JUL-19,2001 - 2002
1,93784,2001,550634200500,Ardsley Union Free School District,School District - Independent Superintendent,Westchester,06/30,A10104,"Legislative Board, Contr Expend",STATEMENT OF EXPENDITURES AND OTHER USES,EXPENDITURES,GENERAL GOVERNMENT,ADMINISTRATION,Contractual,11440.0,31-JUL-19,2001 - 2002
2,93785,2001,550634200500,Ardsley Union Free School District,School District - Independent Superintendent,Westchester,06/30,A101045,Leg Board Supplies & Material,STATEMENT OF EXPENDITURES AND OTHER USES,EXPENDITURES,GENERAL GOVERNMENT,ADMINISTRATION,Contractual,5401.0,31-JUL-19,2001 - 2002
3,93786,2001,550634200500,Ardsley Union Free School District,School District - Independent Superintendent,Westchester,06/30,A101049,Bd of Ed Boces Services,STATEMENT OF EXPENDITURES AND OTHER USES,EXPENDITURES,GENERAL GOVERNMENT,ADMINISTRATION,Contractual,238.0,31-JUL-19,2001 - 2002
4,93787,2001,550634200500,Ardsley Union Free School District,School District - Independent Superintendent,Westchester,06/30,A104016,Noninstructional Salaries,STATEMENT OF EXPENDITURES AND OTHER USES,EXPENDITURES,GENERAL GOVERNMENT,ADMINISTRATION,Personal Services,5500.0,31-JUL-19,2001 - 2002


In [13]:
df = df[[
    'Year',
    'Fiscal Year',
    'Municipal Code',
    'Entity Name',
    'Class Description',
    'County',
    'Account Code',
    'Account Code Narrative',
    'Financial Statement',
    'Financial Statement Segment',
    'Level 1 Category',
    'Level 2 Category',
    'Object of Expenditure',
    'Amount',
]]

KeyError: "['Entity Name'] not in index"

In [ ]:
df.head()

In [ ]:
df.to_csv(output_file, index=False)

In [ ]:
df['Level 2 Category'].value_counts()